In [ ]:
# EfficientNet をインストール
!pip install -U git+https://github.com/qubvel/efficientnet

In [ ]:
# ライブラリの読み込み

import os
import sys
import random
import shutil
import gc
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import keras
import keras.models as M
import keras.layers as L
import keras.backend as K
import tensorflow as tf

from skimage.io import imread
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.constraints import max_norm
from keras.applications.imagenet_utils import decode_predictions
from PIL import Image, ImageEnhance, ImageOps
from efficientnet.keras import EfficientNetB0
from efficientnet.keras import EfficientNetB7
from efficientnet.keras import center_crop_and_resize, preprocess_input
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

In [ ]:
# ディレクトリ移動
%cd drive/MyDrive/mokumoku/dataset

In [ ]:
# tsvファイルの読み込み
train_master = pd.read_csv('train_master.tsv', sep='\t')
label_master = pd.read_csv('label_master.tsv', sep='\t')

master = pd.merge(train_master, label_master, on='label_id', how='inner')
sample = pd.read_csv("sample_submit.csv", header=None, sep=",")

In [ ]:
# このコードを実行するカレントディレクトリは"../inputs"を想定
"""
DATADIR = "inputs"
training_data = []
test_data = []

def create_data(data):
    path = os.path.join(DATADIR, data)
    if data == 'train':
        for image_name, class_num, _ in master.values:
            print(image_name)
            img_array = img_to_array(load_img(os.path.join(path, image_name)))  # 画像読み込み
            training_data.append([img_array, class_num])  # 画像データ、ラベル情報を追加
    if data == 'test':
        for image_name in sample[0].values:
            img_array = img_to_array(load_img(os.path.join(path, image_name))) # 画像読み込み            
            test_data.append(img_array)  # 画像データ、ラベル情報を追加
            
create_data('train')
create_data('test')

# 訓練データと検証データに8:2で分割
train_data, valid_data = train_test_split(training_data, shuffle=True, test_size=0.2, random_state=42)

# データセットの作成
X_train = []  # 画像データ
y_train = []  # ラベル情報

# データセット作成
for feature, label in train_data:
    X_train.append(feature)
    y_train.append(label)
# numpy配列に変換
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

X_valid = []  # 画像データ
y_valid = []  # ラベル情報

# データセット作成
for feature, label in valid_data:
    X_valid.append(feature)
    y_valid.append(label)
# numpy配列に変換
X_valid = np.asarray(X_valid)
y_valid = np.asarray(y_valid)

# numpy配列に変換
test_data = np.asarray(test_data)
"""

In [ ]:
# .npyよりデータを読み込み
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
X_valid = np.load('X_valid.npy')
y_valid = np.load('y_valid.npy')
test_data = np.load('test_data.npy')

In [ ]:
class Rand_Augment():
    def __init__(self, Numbers=None, max_Magnitude=None):
        self.transforms = ['autocontrast', 'equalize', 'rotate', 'solarize', 'color', 'posterize',
                           'contrast', 'brightness', 'sharpness', 'shearX', 'shearY', 'translateX', 'translateY']
        if Numbers is None:
            self.Numbers = len(self.transforms) // 2
        else:
            self.Numbers = Numbers
        if max_Magnitude is None:
            self.max_Magnitude = 10
        else:
            self.max_Magnitude = max_Magnitude
        fillcolor = 128
        self.ranges = {
            # these  Magnitude   range , you  must test  it  yourself , see  what  will happen  after these  operation ,
            # it is no  need to obey  the value  in  autoaugment.py
            "shearX": np.linspace(0, 0.3, 10),
            "shearY": np.linspace(0, 0.3, 10),
            "translateX": np.linspace(0, 0.2, 10),
            "translateY": np.linspace(0, 0.2, 10),
            "rotate": np.linspace(0, 360, 10),
            "color": np.linspace(0.0, 0.9, 10),
            "posterize": np.round(np.linspace(8, 4, 10), 0).astype(np.int),
            "solarize": np.linspace(256, 231, 10),
            "contrast": np.linspace(0.0, 0.5, 10),
            "sharpness": np.linspace(0.0, 0.9, 10),
            "brightness": np.linspace(0.0, 0.3, 10),
            "autocontrast": [0] * 10,
            "equalize": [0] * 10,           
            "invert": [0] * 10
        }
        self.func = {
            "shearX": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, magnitude * random.choice([-1, 1]), 0, 0, 1, 0),
                Image.BICUBIC, fill=fillcolor),
            "shearY": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, 0, magnitude * random.choice([-1, 1]), 1, 0),
                Image.BICUBIC, fill=fillcolor),
            "translateX": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, magnitude * img.size[0] * random.choice([-1, 1]), 0, 1, 0),
                fill=fillcolor),
            "translateY": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, 0, 0, 1, magnitude * img.size[1] * random.choice([-1, 1])),
                fill=fillcolor),
            "rotate": lambda img, magnitude: self.rotate_with_fill(img, magnitude),
            # "rotate": lambda img, magnitude: img.rotate(magnitude * random.choice([-1, 1])),
            "color": lambda img, magnitude: ImageEnhance.Color(img).enhance(1 + magnitude * random.choice([-1, 1])),
            "posterize": lambda img, magnitude: ImageOps.posterize(img, magnitude),
            "solarize": lambda img, magnitude: ImageOps.solarize(img, magnitude),
            "contrast": lambda img, magnitude: ImageEnhance.Contrast(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "sharpness": lambda img, magnitude: ImageEnhance.Sharpness(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "brightness": lambda img, magnitude: ImageEnhance.Brightness(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "autocontrast": lambda img, magnitude: ImageOps.autocontrast(img),
            "equalize": lambda img, magnitude: img,
            "invert": lambda img, magnitude: ImageOps.invert(img)
        }

    def rand_augment(self):
        """Generate a set of distortions.
             Args:
             N: Number of augmentation transformations to apply sequentially. N  is len(transforms)/2  will be best
             M: Max_Magnitude for all the transformations. should be  <= self.max_Magnitude """

        M = np.random.randint(0, self.max_Magnitude, self.Numbers)

        sampled_ops = np.random.choice(self.transforms, self.Numbers)
        return [(op, Magnitude) for (op, Magnitude) in zip(sampled_ops, M)]

    def __call__(self, image):
        operations = self.rand_augment()
        for (op_name, M) in operations:
            operation = self.func[op_name]
            mag = self.ranges[op_name][M]
            image = operation(image, mag)
        return image

    def rotate_with_fill(self, img, magnitude):
        #  I  don't know why  rotate  must change to RGBA , it is  copy  from Autoaugment - pytorch
        rot = img.convert("RGBA").rotate(magnitude)
        return Image.composite(rot, Image.new("RGBA", rot.size, (128,) * 4), rot).convert(img.mode)

    def test_single_operation(self, image, op_name, M=-1):
        '''
        :param image: image
        :param op_name: operation name in   self.transforms
        :param M: -1  stands  for the  max   Magnitude  in  there operation
        :return:
        '''
        operation = self.func[op_name]
        mag = self.ranges[op_name][M]
        image = operation(image, mag)
        return image

In [ ]:
def get_random_data(x_train_i, y_train_i, data_aug):
    x = array_to_img(x_train_i)

    if data_aug:

        seed_image = img_augment(x)
        seed_image = img_to_array(seed_image)

    else:
        seed_image = x_train_i

    seed_image = seed_image / 255

    return seed_image, y_train_i

def data_generator(x_train, y_train, batch_size, data_aug):
    '''data generator for fit_generator'''
    n = len(x_train)
    i = 0
    while True:
        image_data = []
        label_data = []
        for b in range(batch_size):
            if i==0:
                p = np.random.permutation(len(x_train))
                x_train = x_train[p]
                y_train = y_train[p]
            image, label = get_random_data(x_train[i], y_train[i], data_aug)
            image_data.append(image)
            label_data.append(label)
            i = (i+1) % n
        image_data = np.array(image_data)
        label_data = np.array(label_data)
        yield image_data, label_data

In [ ]:
# 学習に時間がかかるため batch_size, lr を高く設定
from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint, EarlyStopping
from efficientnet.keras import EfficientNetB0
from keras.models import Sequential

batch_size = 512
n_classes = 20
steps_per_epoch = X_train.shape[0] // batch_size
validation_steps = X_valid.shape[0] // batch_size
y_train_categorical = keras.utils.to_categorical(y_train)
y_valid_categorical = keras.utils.to_categorical(y_valid)
sgd = optimizers.SGD(lr=0.1, decay=1e-4, momentum=0.9, nesterov=True)

In [ ]:
# RandAugumentによる最適な特徴量の探索
grid_num = 5
grid_magnitude = 10
result = pd.DataFrame(index=range(1, 6), columns=range(1, 11))


for num in range(1, grid_num+1):
    for magnitude in range(1, grid_magnitude+1):
        img_augment = Rand_Augment(Numbers=num, max_Magnitude=magnitude)

        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

        # モデルの構築
        INPUT_SHAPE = (32, 32, 3)
        """
        base_model = EfficientNetB7(input_shape=INPUT_SHAPE, weights='imagenet', include_top=False)
        x = keras.layers.GlobalAveragePooling2D()(base_model.output)
        output = keras.layers.Dense(n_classes, activation='softmax')(x)
        model = keras.models.Model(inputs=[base_model.input], outputs=[output])
        """

        es_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')
        tb_cb = keras.callbacks.TensorBoard(log_dir='output', histogram_freq=1)

        efnb7 = EfficientNetB7(weights = 'imagenet', include_top = False, classes = n_classes, input_shape = INPUT_SHAPE)

        model = Sequential()
        model.add(efnb7)
        model.add(keras.layers.GlobalAveragePooling2D())
        # model.add(keras.layers.Dropout(0.2))
        model.add(keras.layers.Dense(n_classes, activation = 'softmax'))
        # efnb0.trainable = False

        model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
        history = model.fit_generator(data_generator(X_train, y_train_categorical, batch_size, data_aug = True),
                                      initial_epoch=0,
                                      epochs=5,
                        
                                      steps_per_epoch = steps_per_epoch,
                                      validation_data = data_generator(X_valid, y_valid_categorical, batch_size, data_aug = False),
                                      validation_steps = validation_steps,
                                      verbose=1,
                                      callbacks=[es_cb, tb_cb])
        result.loc[num, magnitude] = min(history.history['val_loss'])
        print("val_loss:{}".format(min(history.history['val_loss'])))
        keras.backend.clear_session()
        gc.collect()

In [ ]:
result